In [9]:
import pandas as pd
import numpy as np
import json

In [10]:
disasters = pd.read_csv('train.csv')

## Eliminando duplicados

In [11]:
def getMostFrequentValue(x):
    return x.value_counts().index[0] if len(x.value_counts()) > 0 else np.NaN

#Para todos los tweets tomamos el valor mas frecuente (o NaN si no hay valor) de las columnas keyword y location indexando por texto
disasters['clean_keyword'] = disasters.loc[:, ['text', 'keyword']].groupby(['text']).transform(getMostFrequentValue)
disasters['clean_location'] = disasters.loc[:, ['text', 'location']].groupby(['text']).transform(getMostFrequentValue)

In [12]:
#Agrego la columna target_mean la cual servira para analizar los casos en que un tweet repetido haya sido tagueado de las dos formas posibles
disasters['target_mean'] = disasters.loc[:, ['text', 'target']].groupby('text').transform('mean')

In [13]:
disasters.iloc[[4232, 4235, 4379, 4381]]['target_mean'] = 1
disasters.iloc[[610, 624, 630, 634, 1197, 1331, 1214, 1365, 1221, 1349, 4306, 4320, 5321, 5641, 6091, 6094, 6103, 6123, 6614, 6616]]['target_mean'] = 0

#1197, 1331 -> cancion The Art Of Parties de Japan: https://www.youtube.com/watch?v=nI3wvK8-D10
#1214, 1365 -> Letra de otra cancion. Tweet completo: https://twitter.com/ARCaliber/status/1189236220888567808
#1221, 1349 -> nota: https://www.loc.gov/item/2018695363/
#4306, 4320 -> Tweet contra el islam
#4379, 4381 -> Tweet en referencia al secuestrador Dan Cooper (secuestro un avion en 1971)
#5320, 5641 -> Nigerianos refugiados repatriados de Camerun
#6091, 6094, 6103, 6123 -> Traduciendo se explica
#6614, 6616 -> Tweet comparando el islam con el terrorismo

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
#Aquellos tweets repetidos que tengan un target_mean menor a 0.5 los consideraremos con target 0 (no desastres)
disasters.loc[disasters['target_mean'] < 0.5, ['target']] = 0

#Aquellos tweets repetidos que tengan un target_mean mayor a 0.5 los consideraremos con target 1 (desastres)
disasters.loc[disasters['target_mean'] > 0.5, ['target']] = 1

In [15]:
disasters['keyword'] = disasters['clean_keyword']
disasters['location'] = disasters['clean_location']

In [16]:
disasters.drop_duplicates(subset='text', keep='last', inplace=True)
disasters.loc[:, ['id', 'keyword', 'location', 'text', 'target']].to_csv('clean_train.csv', index=False)

## Generando las clean locations y agregandolas al clean_train

In [ ]:
import requests
import json

In [ ]:
locs = disasters['location'].value_counts().reset_index()
locs.loc[locs['location'] > 1].sort_values(by='location', ascending=False).head(100)
locs.columns = ['location', 'qty']
locs['clean_location'] = 'undefined'

In [ ]:
#import requests
#locs['google_response'] = locs['location'].apply(lambda l: requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + l + '&key=APIKEY').text)

In [ ]:
def findCountry(address_components):
    for comp in address_components:
        if (len(comp['types']) > 0 and comp['types'][0] == 'country'):
            return comp['long_name']
        
        if (len(comp['types']) > 0 and comp['types'][0] == 'establishment'):
            return comp['long_name']
        
    return "undefined"

def findCity(address_components):
    for comp in address_components:
        if (len(comp['types']) > 0 and comp['types'][0] == 'locality'):
            return comp['long_name']
    
    return "undefined"

def findLat(geometry):
    return geometry['location']['lat']

def findLon(geometry):
    return geometry['location']['lng']

class JSONObject(object):
    def __init__(self, data):
        self.__dict__ = json.loads(data)

In [ ]:
def parseAndFindCity(x):
    jObject = JSONObject(x)
    if (len(jObject.results) > 0):
        return findCity(jObject.results[0]['address_components'])
    
    return "undefined"

def parseAndFindCountry(x):
    jObject = JSONObject(x)
    if (len(jObject.results) > 0):
        return findCountry(jObject.results[0]['address_components'])
    
    return "undefined"

def parseAndFindLat(x):
    jObject = JSONObject(x)
    if (len(jObject.results) > 0):
        return findLat(jObject.results[0]['geometry'])
    
    return np.NaN

def parseAndFindLon(x):
    jObject = JSONObject(x)
    if (len(jObject.results) > 0):
        return findLon(jObject.results[0]['geometry'])
    
    return np.NaN

locs['clean_country'] = locs['google_response'].apply(parseAndFindCountry)
locs['clean_city'] = locs['google_response'].apply(parseAndFindCity)
locs['lat'] = locs['google_response'].apply(parseAndFindLat)
locs['lon'] = locs['google_response'].apply(parseAndFindLon)

In [ ]:
locs.loc[:, ['location', 'google_response', 'clean_country', 'clean_city']].to_csv('clean_locations.csv', index=False)

In [17]:
locs = pd.read_csv('clean_locations.csv')
merged = disasters.merge(locs.loc[:, ['location', 'clean_country', 'clean_city', 'lat', 'lon']], how='left', on='location')
merged[['clean_country', 'clean_city']] = merged[['clean_country', 'clean_city']].fillna(value='undefined')
merged.to_csv('clean_train.csv', index=False)